In [2]:
from data.cifar10.plcifar10 import CIFAR10Data

In [3]:
from config import CFG

In [4]:
CFG.__dict__

mappingproxy({'__module__': 'config',
              '__doc__': '_summary_',
              'seed': 43,
              'data_dir': './data/cifar10',
              'pth_dir': './pth',
              'fold': 5,
              'n_split': 5,
              'num_workers': 2,
              'MODEL': 'final',
              'amp': True,
              'print_freq': 100,
              'batch_size': 128,
              'image_size': 128,
              'epochs': 10,
              'patience': 3,
              'accum_iter': 1,
              'max_grad_norm': 1000,
              'max_lr': 0.01,
              'min_lr': 1e-05,
              'weight_decay': 1e-06,
              'augmix': 1,
              'tta': 3,
              'weights': [1],
              '__dict__': <attribute '__dict__' of 'CFG' objects>,
              '__weakref__': <attribute '__weakref__' of 'CFG' objects>})

In [5]:
cifar = CIFAR10Data(CFG.data_dir, CFG.batch_size, 0)

In [6]:
cifar.prepare_data()
cifar.setup(stage='fit')

In [7]:
from model.alexnet import AlexNet, AlexNetLit

In [8]:
CFG.min_lr

1e-05

In [9]:
model = AlexNetLit(10,lr=CFG.min_lr)

In [10]:
from pytorch_lightning.loggers import WandbLogger
wandb_logger = WandbLogger(name='test1',project='image_playground')

wandb: Currently logged in as: surdarla. Use `wandb login --relogin` to force relogin


In [11]:
import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
pl.seed_everything(43)
trainer = pl.Trainer(
    max_epochs=10,
    accelerator="auto",
    devices=1 if torch.cuda.is_available() else None,  # limiting got iPython runs
    logger=wandb_logger,
    log_every_n_steps=50,
    callbacks=[EarlyStopping(monitor="VALIDATION LOSS", mode="min")],
)

Global seed set to 43
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model,cifar)


  | Name      | Type             | Params
-----------------------------------------------
0 | model     | AlexNet          | 56.8 M
1 | loss      | CrossEntropyLoss | 0     
2 | train_acc | Accuracy         | 0     
3 | valid_acc | Accuracy         | 0     
4 | test_acc  | Accuracy         | 0     
-----------------------------------------------
56.8 M    Trainable params
0         Non-trainable params
56.8 M    Total params
227.307   Total estimated model params size (MB)


1e-05


Sanity Checking: 0it [00:00, ?it/s]

/Users/aljam/opt/anaconda3/envs/image_play/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


In [2]:
from config import CFG
CFG.__dict__

mappingproxy({'__module__': 'config',
              '__doc__': '_summary_',
              'seed': 43,
              'data_dir': './data/cifar10',
              'pth_dir': './pth',
              'fold': 5,
              'n_split': 5,
              'num_workers': 2,
              'MODEL': 'final',
              'amp': True,
              'print_freq': 100,
              'batch_size': 128,
              'image_size': 128,
              'epochs': 10,
              'patience': 3,
              'accum_iter': 1,
              'max_grad_norm': 1000,
              'max_lr': 0.01,
              'min_lr': 1e-05,
              'weight_decay': 1e-06,
              'augmix': 1,
              'tta': 3,
              'weights': [1],
              '__dict__': <attribute '__dict__' of 'CFG' objects>,
              '__weakref__': <attribute '__weakref__' of 'CFG' objects>})

In [4]:
from typing import Dict, List, Union
cfgs: Dict[str, List[Union[str, int]]] = {
    'VGG11' : [64, 'M', 128, 'M', 256, 256, 'M', 512,512, 'M',512,512,'M'],
    'VGG13' : [64,64, 'M', 128, 128, 'M', 256, 256, 'M', 512,512, 'M', 512,512,'M'],
    'VGG16' : [64,64, 'M', 128, 128, 'M', 256, 256,256, 'M', 512,512,512, 'M',512,512,512,'M'],
    'VGG19' : [64,64, 'M', 128, 128, 'M', 256, 256,256,256, \
                        'M', 512,512,512,512, 'M',512,512,512,512,'M']
    }

In [5]:
type(cfgs['VGG11'])

list

In [ ]:
from model.vgg import VGG

In [ ]:
VGG('VGG13',batch_norm=True)

In [ ]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger
import wandb

from data.cifar10.plcifar10 import CIFAR10Data
from config import CFG
# from model.alexnet import AlexNetLit
from model.vgg import VGG

wandb.login(key=CFG.wandb_key)
cifar = CIFAR10Data(CFG.data_dir, CFG.batch_size, 0)
cifar.prepare_data()
cifar.setup(stage="fit")
model = VGG()
wandb_logger = WandbLogger(name="test1", project="image_playground")
pl.seed_everything(43)
trainer = pl.Trainer(
    max_epochs=100,
    accelerator="auto",
    auto_lr_find=True,
    auto_scale_batch_size=True,
    devices=1 if torch.cuda.is_available() else None,  # limiting got iPython runs
    logger=wandb_logger,
    log_every_n_steps=50,
    callbacks=[
        EarlyStopping(monitor="VALID LOSS", mode="min", patience=5, verbose=True),
        LearningRateMonitor(logging_interval='step'),
    ],
)
trainer.fit(model, cifar)
trainer.test(model, cifar)
trainer.save_checkpoint(CFG.pth_dir, f"{CFG.MODEL}.pth")
wandb.finish()


wandb: Currently logged in as: surdarla. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/aljam/.netrc


Global seed set to 43
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name        | Type              | Params
--------------------------------------------------
0 | loss        | CrossEntropyLoss  | 0     
1 | train_acc   | Accuracy          | 0     
2 | valid_acc   | Accuracy          | 0     
3 | test_acc    | Accuracy          | 0     
4 | conv_layers | Sequential        | 20.0 M
5 | avgpool     | AdaptiveAvgPool2d | 0     
6 | classifier  | Sequential        | 119 M 
--------------------------------------------------
139 M     Trainable params
0         Non-trainable params
139 M     Total params
558.489   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/aljam/opt/anaconda3/envs/image_play/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/aljam/opt/anaconda3/envs/image_play/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

/Users/aljam/opt/anaconda3/envs/image_play/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
/Users/aljam/opt/anaconda3/envs/image_play/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]